In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/02/26 18:34:18 WARN Utils: Your hostname, KUMAR-PRAKHAR resolves to a loopback address: 127.0.1.1; using 172.24.78.113 instead (on interface eth0)
23/02/26 18:34:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/26 18:34:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Read Data

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```sql
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

### Filter Data

In [4]:
from datetime import datetime

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

### Prepare for GroupBy

In [6]:
rows = rdd.take(10)
row = rows[0]

In [7]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 27, 23, 23, 13), PULocationID=260, total_amount=16.0)

In [8]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

### Aggregate Data

In [9]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

### Unwrap Data

In [10]:
from collections import namedtuple

In [11]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [12]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

### Schema

In [13]:
from pyspark.sql import types

In [14]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

### Result

In [15]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [16]:
df_result.write.parquet('tmp/green-revenue', mode='overwrite')

23/02/26 18:34:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### Map Partition

In [17]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

import pyspark.pandas as ps

In [18]:
import pandas as pd

In [19]:
rows = duration_rdd.take(10)

In [20]:
df = pd.DataFrame(rows, columns=columns)

### Demo Model

In [21]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [22]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

### Map Partition

In [24]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .toDF() \
    .drop('Index')

In [25]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|              14.9|
|             38.25|
|              9.15|
|              8.25|
|              31.9|
|              52.0|
|             14.75|
|               5.0|
|               2.0|
|              4.55|
|              10.1|
|              18.0|
|3.3000000000000003|
|               9.0|
|              13.5|
|              12.9|
|24.950000000000003|
|22.450000000000003|
|10.149999999999999|
|              18.5|
+------------------+
only showing top 20 rows



### END